# Hand tracking

In [2]:
import cv2
import mediapipe as mp
import time
from datetime import datetime

In [3]:
class handDetector():
    def __init__(self, mode = False, maxHands = 6, modelComplex = 1, detectionCon = 0.5, trackCon = 0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.modelComplex = modelComplex
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.modelComplex, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        
    def findHands(self,img, draw = True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

#     def findPosition(self, img, handNo = 0, draw = True):

#         lmlist = []
#         hand_count = 0
#         if self.results.multi_hand_landmarks:
#             for hand in self.results.multi_hand_landmarks:
#                 hand_count += 1
#             myHand = self.results.multi_hand_landmarks[hand_count-1]
#             for id, lm in enumerate(myHand.landmark):
#                 h, w, c = img.shape
#                 cx, cy = int(lm.x * w), int(lm.y * h)
#                 lmlist.append([id, cx, cy])
#                 if draw:
#                     cv2.circle(img, (cx, cy), 3, (255, 0, 255), cv2.FILLED)
#         return lmlist, hand_count
    
    def findPosition(self, img, handNo = 0, draw = True):

        all_lmlist = []
        hand_count = 0
        if self.results.multi_hand_landmarks:
            for hand in self.results.multi_hand_landmarks:
                hand_count += 1
                lmlist = []
                myHand = self.results.multi_hand_landmarks[hand_count-1]
                for id, lm in enumerate(myHand.landmark):
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    lmlist.append([id, cx, cy])
                    if draw:
                        cv2.circle(img, (cx, cy), 3, (255, 0, 255), cv2.FILLED)
                all_lmlist.append(lmlist)
        return all_lmlist, hand_count


In [3]:
pTime = 0
cTime = 0
cap = cv2.VideoCapture(0)
detector = handDetector()

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    all_lmlist, hand_count = detector.findPosition(img)
    if len(all_lmlist) != 0:
        print("timestamp:", datetime.now().time())
#         print(all_lmlist)
        print("number of hands detected:", hand_count)
        for lmlist in all_lmlist:
            print("hand coordinates", [lmlist[4][1], lmlist[4][2]])
        print("\n")

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 3)

    cv2.imshow("Image", img)
    cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


timestamp: 10:47:13.938131
number of hands detected: 1
hand coordinates [707, 522]


timestamp: 10:47:14.031827
number of hands detected: 2
hand coordinates [445, 447]
hand coordinates [710, 498]


timestamp: 10:47:14.134355
number of hands detected: 2
hand coordinates [475, 456]
hand coordinates [698, 507]


timestamp: 10:47:14.227265
number of hands detected: 2
hand coordinates [499, 468]
hand coordinates [690, 512]


timestamp: 10:47:14.332419
number of hands detected: 2
hand coordinates [521, 477]
hand coordinates [688, 501]


timestamp: 10:47:14.427098
number of hands detected: 2
hand coordinates [537, 461]
hand coordinates [675, 493]


timestamp: 10:47:14.494177
number of hands detected: 2
hand coordinates [550, 453]
hand coordinates [673, 481]


timestamp: 10:47:14.564754
number of hands detected: 2
hand coordinates [555, 445]
hand coordinates [666, 476]


timestamp: 10:47:14.662277
number of hands detected: 2
hand coordinates [565, 442]
hand coordinates [649, 470]


timestamp: 

KeyboardInterrupt: 

In [ ]:
# pTime = 0
# cTime = 0
# cap = cv2.VideoCapture(0)
# detector = handDetector()

# while True:
#     success, frame = cap.read()
#     img = detector.findHands(frame)
#     all_lmlist, hand_count = detector.findPosition(img)
#     if len(all_lmlist) != 0:
#         print("timestamp:", datetime.now().time())
# #         print(all_lmlist)
#         print("number of hands detected:", hand_count)
#         for lmlist in all_lmlist:
#             print("hand coordinates", [lmlist[4][1], lmlist[4][2]])
#         print("\n")

#     cTime = time.time()
#     fps = 1 / (cTime - pTime)
#     pTime = cTime
    
#     cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 3)

#     cv2.imshow("Image", img)
#     cv2.waitKey(1)

In [ ]:
# cap = cv2.VideoCapture(0)
# hand_detector = handDetector()
# while True:
#     success, img = cap.read()
#     img = hand_detector.findHands(img)
#     cv2.imshow("Image", img)
#     cv2.waitKey(1)

# Ball Detector

In [4]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time

In [5]:
vs = VideoStream(src=0).start()
cap = cv2.VideoCapture(0)
pTime = 0
cTime = 0
hand_detector = handDetector()
purpleLower = (128,0,128)
purpleUpper = (216,191,216)
yellowLower = (22, 93, 0)
yellowUpper = (45, 255, 255)
pts = deque(maxlen=5)
while True:
    # grab the current frame
    success, frame = cap.read()
    img = hand_detector.findHands(frame)
#     cv2.imshow("Image", img)
#     cv2.waitKey(1)
    
    if frame is None:
        break
    # resize the frame, blur it, and convert it to the HSV
    # color space
    frame = imutils.resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    # construct a mask for the color "purple", then perform
    # a series of dilations and erosions to remove any small
    # blobs left in the mask
    mask1 = cv2.inRange(hsv, purpleLower, purpleUpper)
    mask2 = cv2.inRange(hsv, yellowLower, yellowUpper)
    mask = cv2.bitwise_or(mask1, mask2)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    # find contours in the mask and initialize the current
    # (x, y) center of the ball
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None

    # only proceed if at least one contour was found
    if len(cnts) > 0:
        # find the largest contour in the mask, then use
        # it to compute the minimum enclosing circle and
        # centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        # only proceed if the radius meets a minimum size
        if radius > 20:
            # draw the circle and centroid on the frame,
            # then update the list of tracked points
            cv2.circle(frame, (int(x), int(y)), int(radius),
                (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)

    # update the points queue
    pts.appendleft(center)

    # loop over the set of tracked points
    for i in range(1, len(pts)):
        # if either of the tracked points are None, ignore
        # them
        if pts[i - 1] is None or pts[i] is None:
            continue

        # otherwise, compute the thickness of the line and
        # draw the connecting lines
        thickness = 1
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)

    all_lmlist, hand_count = hand_detector.findPosition(img)
        
    if len(all_lmlist) != 0:
        print("timestamp:", datetime.now().time())
#         print(all_lmlist)
        print("number of hands detected:", hand_count)
        for lmlist in all_lmlist:
            print("hand coordinates", [lmlist[4][1], lmlist[4][2]])
        print("\n")

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 3)

    cv2.imshow("Image", frame)
#     cv2.waitKey(1)
#     # show the frame to our screen
#     cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
   
    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break
    
    

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Accelerometer Data